In [1]:
import pandas as pd
import numpy as np
import pdfplumber
import spacy
import re
from wordcloud import WordCloud, STOPWORDS

In [2]:
# (1) Read PDF and Convert It into Excel
def drop_line_breaks(df: pd.DataFrame):
    """
    Function for adjusting the format of the dataframes which PDFPlumber read
    :param df: pd.dataframe needing adjusting
    :return: dataframe after adjusting
    """
    df['成果名称'] = df['成果名称'].apply(lambda x: x.replace('\n', ''))
    df['主要完成人'] = df['主要完成人'].apply(lambda x: x.replace('\n', ''))
    df['主要完成单位'] = df['主要完成单位'].apply(lambda x: x.replace('\n', ''))
    df = df.reindex(columns=['序号', '成果等级', '成果名称', '主要完成人', '主要完成单位'])
    return df


# Best Awards
with pdfplumber.open('./2022教学成果奖公示（本科）.pdf') as pdf:
    page = pdf.pages[0]
    table = page.extract_tables()
best_awards = pd.DataFrame(table[0][1:], columns=table[0][0])
best_awards['成果等级'] = '特等奖'
best_awards = drop_line_breaks(best_awards)


# Gold Awards
gold_awards = pd.DataFrame(table[1][1:], columns=table[1][0])
for i in range(1, 8):
    with pdfplumber.open('./2022教学成果奖公示（本科）.pdf') as pdf:
        page = pdf.pages[i]
        table = page.extract_table()
    gold_awards = pd.concat([gold_awards, pd.DataFrame(table[1:], columns=table[0])])
gold_awards['成果等级'] = '一等奖'
gold_awards = drop_line_breaks(gold_awards)


# Silver Awards
silver_awards = pd.DataFrame()
for i in range(8, 59):
    with pdfplumber.open('./2022教学成果奖公示（本科）.pdf') as pdf:
        page = pdf.pages[i]
        table = page.extract_table()
    silver_awards = pd.concat([silver_awards, pd.DataFrame(table[1:], columns=table[0])])
silver_awards['成果等级'] = '二等奖'
silver_awards = drop_line_breaks(silver_awards)


# Concat and export
awards = pd.concat([best_awards, gold_awards, silver_awards])
awards.to_excel('./2022 年高等教育（本科）国家级教学成果奖拟授奖成果公示名单.xlsx', index=False)

In [4]:
# (2) Export Word Frequency and Wordcloud
# Tokenize
awards_names = ' '.join(awards['成果名称'])
nlp_zh = spacy.load('zh_core_web_lg')
doc = nlp_zh(awards_names)
tokens = []
for token in doc:
    tokens.append(token.text)
tokens = [token for token in tokens if re.compile(r'[\u4e00-\u9fa5]+').match(token)]


# Draw Wordcloud
wordcloud = WordCloud(font_path='msyh.ttc',
                      width=800,
                      height=800,
                      background_color="white").generate(" ".join(tokens))
wordcloud.to_file("wordcloud.jpg")


# Export the Frequency
words = pd.DataFrame(tokens, columns=['word']).reset_index()
words = words.groupby('word')['index'].agg('count').reset_index().sort_values('index', ascending=False).head(50)
words['词汇'] = words['word']
words['词频'] = words['index']
words = words.drop(['word', 'index'], axis=1)
words.to_csv('排名前50词汇词频.txt', index=False)

In [5]:
# (3) Stat the Awards by Universities
awards['主要完成单位'] = awards['主要完成单位'].str.split(',')
awards['完成单位数量'] = awards['主要完成单位'].apply(lambda x: len(x))
awards['得分'] = awards['成果等级'].apply(lambda x: np.select([x == '特等奖', x == '一等奖', x == '二等奖'], [50, 30, 20]))
awards = awards.explode('主要完成单位')
awards['各校得分'] = awards['得分']/awards['完成单位数量']
awards['特等奖'] = np.where(awards['成果等级'] == '特等奖', 1, 0)
awards['一等奖'] = np.where(awards['成果等级'] == '一等奖', 1, 0)
awards['二等奖'] = np.where(awards['成果等级'] == '二等奖', 1, 0)
schools = pd.DataFrame()
schools['特等奖数'] = awards.groupby('主要完成单位')['特等奖'].agg('sum')
schools['一等奖数'] = awards.groupby('主要完成单位')['一等奖'].agg('sum')
schools['二等奖数'] = awards.groupby('主要完成单位')['二等奖'].agg('sum')
schools['得分'] = awards.groupby('主要完成单位')['各校得分'].agg('sum')
schools = schools.reset_index()
schools.to_excel('各完成单位获奖情况.xlsx', index=False)

In [6]:
# (4) Stat the Awards by Regions
school_list = pd.read_excel('./全国高校名单（2022）.xls')
school_list = school_list.drop('Unnamed: 6', axis=1)
school_list = school_list.dropna(axis=0)
school_list.columns = school_list.iloc[0]
school_list = school_list[1:]
schools = schools.merge(school_list, left_on='主要完成单位', right_on='学校名称', how='left')
regions = pd.DataFrame()
regions['特等奖数'] = schools.groupby('所在地')['特等奖数'].agg('sum')
regions['一等奖数'] = schools.groupby('所在地')['一等奖数'].agg('sum')
regions['二等奖数'] = schools.groupby('所在地')['二等奖数'].agg('sum')
regions['得分'] = schools.groupby('所在地')['得分'].agg('sum')
regions = regions.reset_index()
regions.to_excel('各省区获奖情况.xlsx', index=False)